In [1]:
import tensorflow as tf
import tensorflow.feature_column as fc
import os
import sys
import matplotlib.pyplot as plt
from IPython.display import clear_output

tf.enable_eager_execution()

In [2]:
models_path = os.path.join(os.getcwd(), 'models')

sys.path.append(models_path)

In [3]:
from official.wide_deep import census_dataset
from official.wide_deep import census_main

census_dataset.download("/tmp/census_data/")

ImportError: No module named 'official'

In [ ]:
train_file = "/tmp/census_data/adult.data"
test_file = "/tmp/census_data/adult.test"

In [3]:
import pandas

train_df = pandas.read_csv(train_file, header = None, names = census_dataset._CSV_COLUMNS)
test_df = pandas.read_csv(test_file, header = None, names = census_dataset._CSV_COLUMNS)

train_df.head()

NameError: name 'train_file' is not defined

In [1]:
def easy_input_function(df, label_key, num_epoches, shuffle, batch_size):
    label = df[label_key]
    ds = tf.data.Dataset.from_tensor_slices((dict(df), label))

    if shuffle:
        ds = ds.shuffle(10000)

    ds = ds.batch(batch_size).repeat(num_epoches)
    return ds

In [2]:
ds = easy_input_function(train_df, label_key='income_bracket', num_epoches=5, shuffle=True, batch_size=10)

NameError: name 'train_df' is not defined

In [4]:
import inspect

print(inspect.getsource(census_dataset.input_fn))

NameError: name 'census_dataset' is not defined

In [5]:
ds = census_dataset.input_fn(train_file, num_epochs=5, shuffle=True, batch_size=10)

NameError: name 'census_dataset' is not defined

In [6]:
age = fc.numeric_column('age')

NameError: name 'fc' is not defined

In [ ]:
fc.input_layer(feature_bactch,[age]).numpy()

In [7]:
classifier = tf.estimator.LinearClassifier(feature_columns=[age])
classifier.train(train_inpf)
result = classifier.evaluate(test_inpf)

clear_output()
print(result)

NameError: name 'tf' is not defined

In [ ]:
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

my_numeric_columns = [age, education_num, capital_gain, capital_loss, hours_per_week]

fc.input_layer(feature_batch, my_numeric_columns).numpy()

In [ ]:
classifier = tf.estimator.LinearClassifier(feature_columns=my_numeric_columns)
classifier.train(train_inpf)

result = classifier.evaluate(test_inpf)

clear_output()

for key, value in sorted(result.items()):
    print('%s: %s' % (key, value))

In [8]:
relationship = fc.categorical_column_with_vocabulary_list('relationship',
                                                          ['Husband', 'Not-in-family', 'wife', 'Own-child',
                                                           'Unmarried', 'Other-relative'])

NameError: name 'fc' is not defined

In [ ]:
fc.input_layer(feature_batch, [age, fc.indicator_column(relationship)])

In [ ]:
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=1000)

In [ ]:
for item in feature_batch['occupation'].numpy():
    print(item.decode())

In [ ]:
occupation_result = fc.input_layer(feature_batch, [fc.indicator_column(occupation)])
occupation_result.numpy().shape

In [ ]:
tf.argmax(occupation_result, axis=1).numpy()

In [9]:
education = tf.feature_column.categorical_column_with_vocabulary_list(
    'education', [
        'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
        '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    'marital_status', [
        'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
        'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    'workclass', [
        'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
        'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

my_categorical_columns = [relationship, occupation, education, marital_status, workclass]

NameError: name 'tf' is not defined

In [ ]:
classifier = tf.estimator.LinearClassifier(feature_columns=my_numeric_columns + my_categorical_columns)
classifier.train(train_inpf)
result = classifier.evaluate(test_inpf)

clear_output()

for key, value in sorted(result.items()):
    print('%s: %s' % (key, value))

In [ ]:
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 60, 65])

In [ ]:
fc.input_layer(feature_batch, [age, age_buckets]).numpy()

In [ ]:
education_x_occupation = tf.feature_column.crossed_column(['education', 'occupation'], hash_bucket_size=1000)

In [ ]:
age_buckets_x_education_x_occupation = tf.feature_column.crossed_column([age_buckets, 'education',
                                                                         'occupaion'], hash_bucket_size=1000)

In [ ]:
import tempfile

base_columns = [
    education, marital_status, relationship, workclass, occupation,
    age_buckets,
]

crossed_columns = [
    tf.feature_column.crossed_column(
        ['education', 'occupation'], hash_bucket_size=1000),
    tf.feature_column.crossed_column(
        [age_buckets, 'education', 'occupation'], hash_bucket_size=1000),
]

model = tf.estimator.LinearClassifier(
    model_dir=tempfile.mkdtemp(),
    feature_columns=base_columns + crossed_columns,
    optimizer=tf.train.FtrlOptimizer(learning_rate=0.1))

In [ ]:
import functools

train_inpf = functools.partial(census_dataset.input_fn, train_file, num_epochs=40, shuffle=True, batch_size=64)

model.train(train_inpf)
clear_output()

In [ ]:
results = model.evaluate(test_inpf)
clear_output()

for key.value in sorted(result.items()):
    print('%s: %0.2f' % (key, value))

In [ ]:
import numpy as np

predict_df = test_df[:20].copy()

pred_iter = model.predict(lambda: easy_input_function(predict_df, label_key='income_bracket',
                                                      num_epoches=1, shuffle=False, batch_size=10))

classes = np.array(['<=50k', '>50k'])
pred_class_id = []

for pred_dict in pred_iter:
    pred_class_id.append(pred_dict['class_ids'])

predict_df['predicted_class'] = classes[np.array(pred_class_id)]
predict_df['correct'] = predict_df['predicted_class'] == predict_df['income_bracker']

clear_output()

predict_df[['income_bracket', 'predicted_class', 'correct']]

In [ ]:
model_l1 = tf.estimator.LinearClassifier(feature_columns=base_columns + crossed_columns,
                                         optimizer=tf.train.FtrlOptimizer(learning_rate=0.1,
                                                                          l1_regularization_strength=10.0,
                                                                          l2_shrinkage_regularization_strength=0.0))

In [ ]:
model_l1.train(train_inpf)
results = model_l1.evaluate(test_inpf)
clear_output()
for key in sorted(results):
    print('%s:$0.2f' % (key, results[key]))